Мы – магазин канцтоваров, выбираем поставщика общих тетрадей, которые будут хорошо продаваться и принесут максимум прибыли. Нужно выбрать только 1 поставщика с несколькими 2-3 наименованиями тетрадей.
Какое поставщик будет наиболее перспективным?

Данные: https://video.ittensive.com/analytics-way/2022-04-05.ittensive.010.xlsx

In [1]:
# подключение библиотек
import pandas as pd

In [2]:
#загрузка данных и преобразование в pandas DataFrame

data = pd.read_excel('https://video.ittensive.com/analytics-way/2022-04-05.ittensive.010.xlsx')
data.head()

,№,шероховатость бумаги (1мин-10макс),"плотность бумаги, г/м2",качество обложки (1мин-10макс),кол-во листов,"опт. цена, руб.",Поставщик
0,1,5,50,10,48,80,A
1,2,4,120,1,48,85,B
2,3,7,100,6,96,100,D
3,4,9,55,7,64,70,A
4,5,5,80,9,48,50,D


Постановка задачи. Выбрать наиболее перспективного поставщика.<br>
Нужно получить максимально качественный товар, за минимальную закупочную стоимость.<br>
Перспективным будет тот, у кого максимальное количество товаров будут обладать максимальным соотношением цена / качество, относительно остальных.

Условно будем отталкиваться от цены за 1 лист и индекса качества тетради.

Некоторые пояснения параметров:
* Шероховатость - лучше мин. значение
* Плотность  - лучше макс. значение
* Качество обложки - с точки зрения маркетинга лучше макс. значение

Введем общий индекс качества тетради который будет включать в себя сумму качества всех параметров, а также предусмотрим некоторый поправочный коэффициент, которым можно регулировать общую оценку, понижая важность какого либо параметра.

In [3]:
coef_roughness = 1  # коэффициент шершавости 
coef_density = 1  # коэффициент плотности 
coef_qualty_cover = 1  # коэффициент качества обложки

Создадим отдельный фрейм данных с относительными величинами качеств параметров тетрадей.

In [4]:
data2 = pd.DataFrame()

In [5]:
data2['Шероховатость бумаги, %'] = data['шероховатость бумаги (1мин-10макс)'].apply(lambda x: 100 - (x/10) * 100 * coef_roughness)
data2['Плотность бумаги, %'] = round(data['плотность бумаги, г/м2'].apply(lambda x: x/data['плотность бумаги, г/м2'].max() * 100 * coef_density), 2)
data2['Качество обложки, %'] = data['качество обложки (1мин-10макс)'].apply(lambda x: (x/10) * 100 * coef_qualty_cover)
data2['Суммарное качество, %'] = round((data2['Шероховатость бумаги, %'] + data2['Плотность бумаги, %'] + data2['Качество обложки, %']) / 3, 2)
data2['Цена за лист, руб.'] = round(data['опт. цена, руб.'] / data['кол-во листов'], 2)
data2['Цена за лист, %'] = round(data2['Цена за лист, руб.'].apply(lambda x: 100 - x/data2['Цена за лист, руб.'].max() * 100), 2)
data2['Цена / качество, %'] = round((data2['Суммарное качество, %'] + data2['Цена за лист, %']) / 2, 2)
data2['Поставщик'] = data['Поставщик']

In [6]:
# отсортируем по убыванию соотношения 'Цена / качество, %'
data2.sort_values(['Цена / качество, %'], ascending=False)

,"Шероховатость бумаги, %","Плотность бумаги, %","Качество обложки, %","Суммарное качество, %","Цена за лист, руб.","Цена за лист, %","Цена / качество, %",Поставщик
9,60.0,76.92,70.0,68.97,0.68,72.8,70.88,B
6,80.0,92.31,50.0,74.10,1.15,54.0,64.05,D
4,50.0,61.54,90.0,67.18,1.04,58.4,62.79,D
7,50.0,61.54,60.0,57.18,1.04,58.4,57.79,C
2,30.0,76.92,60.0,55.64,1.04,58.4,57.02,D
3,10.0,42.31,70.0,40.77,1.09,56.4,48.58,A
0,50.0,38.46,100.0,62.82,1.67,33.2,48.01,A
8,20.0,69.23,40.0,43.08,1.46,41.6,42.34,C
1,60.0,92.31,10.0,54.10,1.77,29.2,41.65,B
5,20.0,100.00,60.0,60.00,2.50,0.0,30.00,A


Добавим в data полученное соотношение цена качество и отсортируем по убыванию.

In [7]:
data['Цена / качество, %'] = data2['Цена / качество, %']
data.sort_values(['Цена / качество, %'], ascending=False)

,№,шероховатость бумаги (1мин-10макс),"плотность бумаги, г/м2",качество обложки (1мин-10макс),кол-во листов,"опт. цена, руб.",Поставщик,"Цена / качество, %"
9,10,4,100,7,96,65,B,70.88
6,7,2,120,5,48,55,D,64.05
4,5,5,80,9,48,50,D,62.79
7,8,5,80,6,48,50,C,57.79
2,3,7,100,6,96,100,D,57.02
3,4,9,55,7,64,70,A,48.58
0,1,5,50,10,48,80,A,48.01
8,9,8,90,4,48,70,C,42.34
1,2,4,120,1,48,85,B,41.65
5,6,8,130,6,32,80,A,30.00


Поставщик D, по соотношению цена / качество, набирает большинство, по большему количеству позиций. 

При помощи подстройки коэффициентов параметров: плотности, шершавости и качество обложки, можно будет более гибко подбирать тетрадь по соотношению цена / качество, если какой из этих параметров не столь важен при выборе тетради. 
